# Interface Demonstration Notebook

A notebook to demonstrate an interface with some of the features of the Probability of Failure Model

Author: gavin.treseder@essentialenergy.com.au

In [1]:
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

import logging
logging.getLogger().setLevel(logging.INFO)

import copy
import multiprocessing as mp

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash

from config import config
from pof.loader.asset_model_loader import AssetModelLoader
from pof import Component, FailureMode, Task
from pof.interface.dashlogger import DashLogger
from pof.interface.layouts import *
from pof.interface.figures import update_condition_fig, update_cost_fig, update_pof_fig, make_inspection_interval_fig_mod

In [2]:
# Quick test to make sure everything is works 
comp = Component.demo()
mcl = make_component_layout(comp)
print(f"Validation Result: {validate_layout(comp, mcl)}")

Validation Result: True


In [3]:
# Turn off logging
logging.getLogger().setLevel(logging.INFO)
config['Load']['on_error_use_default'] ='False'

file_name = r"\data\inputs\Asset Model - Pole - Demo.xlsx"
# file_name = r"\data\inputs\Asset Model - Pole - Timber.xlsx"
# file_name = r'\data\inputs\Asset Model - Demo.xlsx'

filename = os.path.dirname(os.getcwd()) + file_name
aml = AssetModelLoader(filename)
comp_data = aml.load()
comp = Component.from_dict(comp_data['pole'])

In [4]:
# TODO compare speed
# %%timeit
# comp.expected_risk_cost_df()

# %%timeit
# comp.expected_risk_cost_df_legacy_method()


In [5]:

# Turn off logging level to speed up implementation
logging.getLogger().setLevel(logging.CRITICAL)
logging.getLogger("werkzeug").setLevel(logging.WARNING)

# Build App
external_stylesheets = [dbc.themes.BOOTSTRAP]
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

pof_sim = copy.copy(comp)
sens_sim = copy.copy(comp)

def layout():
    mcl = make_component_layout(comp)
    update_list = [{"label": option, "value": option} for option in comp.get_update_ids()]

    layout = html.Div([
        html.Div(id='log'),
        html.Div(children='Update State:', id='update_state'),
        html.Div([
            html.P(children='Sim State', id='sim_state'),
            html.P(id='sim_state_err', style={'color': 'red'}),
        ]),
        html.Div(children='Fig State', id='fig_state'),
        html.P(id='ffcf'),
        html.Div([
            dcc.Interval(id="progress-interval", n_intervals=0, interval=50),
            dbc.Row([
                dbc.Col([
                    dbc.InputGroupAddon(
                        [
                            dbc.Checkbox(
                                id="sim_n_active", checked=True
                            ),
                            dcc.Input(id='n_iterations-input', value=10, type='number'),
                        ],
                        addon_type="prepend",
                    ),
                    dbc.Progress(id="n-progress"),
                ]),
                dbc.Col([
                    dbc.InputGroupAddon(
                        [
                            dbc.Checkbox(
                                id="sim_n_sens_active", checked=False
                            ),
                            dcc.Input(id='n_sens_iterations-input', value=10, type='number'),
                        ],
                        addon_type="prepend",
                    ),
                    dbc.Progress(id="n_sens-progress"),
                ]),
            ]),
            
        ]),
        dbc.Row([dbc.Col(), dbc.Col(dcc.Dropdown(
            id='demo-dropdown',
            options=update_list,
            value=comp.get_update_ids()[-1]))]),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id='pof-fig')),
                dbc.Col(dcc.Graph(id='insp_interval-fig')),
            ]
        ),
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id='cond-fig')),
                dbc.Col(dcc.Graph(id="cost-fig"))
            ]
        ),
        mcl,
    ])
    
    return layout

# Layout
var_to_scale = cf.scaling
app.layout = layout

collapse_ids = comp.get_objects()

@app.callback(
    [Output(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
    [Input(f"{prefix}-collapse-button", "n_clicks") for prefix in collapse_ids],
    [State(f"{prefix}-collapse", "is_open") for prefix in collapse_ids],
)
def toggle_collapses(*args):
    ctx = dash.callback_context

    state_id = ""
    collapse_id = ctx.triggered[0]['prop_id'].split('.')[0].replace('-collapse-button','')
    if collapse_id in collapse_ids: #TODO change to is not None

        state_id = collapse_id + '-collapse.is_open'
        ctx.states[state_id] = not ctx.states[state_id]

    is_open = tuple(ctx.states.values())

    return is_open


ms_fig_update = comp.get_dash_ids()
param_inputs = [Input(dash_id,"checked") if 'active' in dash_id else Input(dash_id,"value") for dash_id in ms_fig_update]

# Update --> Simulate --> Figures

@app.callback(
    Output('update_state', 'children'), 
    param_inputs
)
def update_parameter(graph_y_limit_active, graph_y_limit, *args):
    """Update a the pof object whenever an input is changes"""

    # Check the parameters that changed
    ctx = dash.callback_context
    dash_id = None
    value=None  

    # If any parameters have changed update the objecte
    if ctx.triggered:
        dash_id = ctx.triggered[0]['prop_id'].split('.')[0]
        value = ctx.triggered[0]['value']

        # Scale the value if req
        value = value / var_to_scale.get(dash_id.split('-')[-1], 1)

        # update the model
        comp.update(dash_id, value)
    
    return f"Update State: {dash_id} - {value}"


@app.callback(
    [Output("sim_state", 'children'), Output("sim_state_err", "children")],
    [Input("sim_n_active", "checked"), Input("n_iterations-input", "value"), Input("update_state", "children")],
    [State("sim_n_active", "checked"), State("n_iterations-input", "value")]
)
def update_simulation(active_input, n_iterations_input, state, active, n_iterations, *args):
    """ Triger a simulation whenever an update is completed or the number of iterations change"""
    global pof_sim
    global sim_err_count

    pof_sim.cancel_sim()
    #time.sleep(1)
    if active:
        pof_sim = copy.copy(comp)

        pof_sim.mp_timeline(t_end=200, n_iterations=n_iterations)

        if not pof_sim.up_to_date:
            sim_err_count = sim_err_count + 1
            return dash.no_update, f"Errors: {sim_err_count}"
        return f"Sim State: {pof_sim.n_iterations} - {n_iterations}", ""
    else:
        return f"Sim State: not updating", ""

fig_start = 0
fig_end = 0

# After a simulation the following callbacks are triggered

@app.callback(
    [Output("cost-fig", "figure"),
    Output("pof-fig", "figure"),
    Output("cond-fig", "figure"),
    Output("fig_state", "children")], 
    [Input("sim_state", "children")],
    [State("sim_n_active", "checked")],
)
def update_figures(state, active, *args):
    if active:

        global fig_start
        global fig_end

        fig_start = fig_start + 1
        cost_fig = update_cost_fig(pof_sim) #legend = dropdown value
        pof_fig = update_pof_fig(pof_sim)
        cond_fig = update_condition_fig(pof_sim)
        fig_end = fig_end + 1
        return cost_fig, pof_fig, cond_fig, f"Fig State: {fig_start} - {fig_end}"
    else:
        raise PreventUpdate

@app.callback(
    Output("ffcf", "children"), 
    [Input("sim_state", "children")]
)
def update_ffcf(*args):
    cf = len(pof_sim.expected_cf())
    ff = len(pof_sim.expected_ff())

    try:
        ratio = round(ff / (cf+ff), 2)
    except:
        ratio = '--.--'

    return f"Conditional {cf} : {ff} Functional. {ratio}%"

@app.callback(
    Output("insp_interval-fig", "figure"), 
    [Input("sim_n_sens_active", "checked"), Input("n_sens_iterations-input", "value"), Input("demo-dropdown", "value"), Input('cost-fig', 'figure')],
    [State("sim_n_sens_active", "checked"), State("n_sens_iterations-input", "value"), State("demo-dropdown", "value")]
)
def update_insp_interval(active_input, n_iterations_input, var_input, fig , active, n_iterations, var_name, *args):
    """ Trigger a sensitivity analysis of the target variable"""
    # Copy from the main model
    global sens_sim
    sens_sim.cancel_sim()

    if active:
        sens_sim = copy.deepcopy(comp)
        
        insp_interval_fig = make_inspection_interval_fig_mod(sens_sim, var_name=var_name, t_min=1, t_max=10, step_size=1, n_iterations = n_iterations)

        return insp_interval_fig
    else:
        raise PreventUpdate

# The following progress bars are always running

@app.callback(
    [Output("n-progress", "value"), Output("n-progress", "children")],
    [Input("progress-interval", "n_intervals")],
)
def update_progress(n):
    if pof_sim.n is None:
        raise Exception("no process started")
    progress = int(pof_sim.progress() * 100)

    return progress, f"{progress} %" if progress >= 5 else ""

@app.callback(
    [Output("n_sens-progress", "value"), Output("n_sens-progress", "children")],
    [Input("progress-interval", "n_intervals")],
)
def update_progress_sens(n):
    if sens_sim.n is None:
        raise Exception("no process started")
    progress = int(sens_sim.sens_progress() * 100)

    return progress, f"{progress} %" if progress >= 5 else ""


#app.run_server(debug=False, mode='inline')
app.run_server(debug=True, use_reloader=False)

Dash app running on http://127.0.0.1:8050/


In [7]:
erc = dict()
for fm_name, fm in comp.fm.items():
    erc[fm_name] = fm._expected_risk_cost()

In [26]:
durations = []
event_observed = []
event='failure'

for timeline in fm._timelines.values():
    event_observed.append(timeline[event].any())
    if event_observed[-1]:
        durations.append(timeline["time"][timeline[event]][0])
    else:
        durations.append(timeline["time"][-1])

In [32]:
        from lifelines import WeibullFitter
        durations = np.array(durations)
        event_observed = np.array(event_observed)
        durations = durations - fm.untreated.gamma

        # Correct for zero times to have occured halfway between the 0 and 0.5
        durations[durations <= 0] = 0.25

        # Fit the weibull
        wbf = WeibullFitter()
        wbf.fit(durations=durations, event_observed=event_observed)

        fm.pof = Distribution(
            alpha=wbf.lambda_,
            beta=wbf.rho_,
            gamma=self.untreated.gamma,
        )

NameError: name 'Distribution' is not defined

In [45]:
event_observed + event_observed

array([False, False, False,  True])

In [47]:
wbf.__dict__

{'alpha': 0.05,
 '_class_name': 'WeibullFitter',
 '_label': 'Weibull_estimate',
 '_censoring_type': <CensoringType.RIGHT: 'right'>,
 '_estimate_name': 'cumulative_hazard_',
 '_bounds': [(1e-09, None), (1e-09, None)],
 'durations': array([38., 48., 51., 68.]),
 'event_observed': array([0, 0, 0, 1]),
 'entry': array([0., 0., 0., 0.]),
 'weights': array([1., 1., 1., 1.]),
 'timeline': array([38., 48., 58., 68.]),
 '_ci_labels': None,
 '_initial_values': array([51.25,  1.  ]),
 '_fitted_parameters_': array([6.80000000e+01, 1.82634904e+05]),
 'log_likelihood_': 6.895736671200532,
 '_hessian_': array([[7.21356142e+06, 0.00000000e+00],
        [0.00000000e+00, 2.99800561e-11]]),
 'lambda_': 68.00000000000004,
 'rho_': 182634.90353349817,
 'variance_matrix_':               lambda_          rho_
 lambda_  1.386278e-07  0.000000e+00
 rho_     0.000000e+00  3.335551e+10,
 'survival_function_':       Weibull_estimate
 38.0          1.000000
 48.0          1.000000
 58.0          1.000000
 68.0    

In [22]:
list(comp.fm['weathering'].expected_risk_cost()['risk'])

['time', 'cost']

In [16]:
df = comp.expected_risk_cost_df().groupby(by=['failure_mode', 'task']).max()
df

time   cost task_active  \
failure_mode            task                                          
fire_damage             CAT1/2 replacement    61      0        True   
                        inspection            61      0        True   
                        risk                  61      0         NaN   
fungal decay _ external CAT1/2 replacement    62    777        True   
                        CAT3/4 replacement    62      0        True   
                        inspection            62     50        True   
                        risk                  62   5555         NaN   
fungal decay _ internal CAT1/2 replacement    62    777        True   
                        CAT3/4 replacement    62      0        True   
                        inspection            62     50        True   
                        pole_saver_rod        62      0        True   
                        risk                  62   5555         NaN   
impact                  CAT1/2 replacement    61      0        True   
                        inspection            61      0        True   
                        risk                  61      0         NaN   
lightning               CAT1/2 replacement    61      0        True   
                        inspection_1          61      0        True   
                        inspection_2          61     50        True   
                        risk                  61      0         NaN   
termites                CAT1/2 replacement    62    777        True   
                        CAT3/4 replacement    62      0        True   
                        inspection            62     50        True   
                        risk                  62   5555         NaN   
                        termite_powder        62      0        True   
weathering              CAT1/2 replacement    55   1555        True   
                        CAT3/4 replacement    55      0        True   
                        inspection            55      0        True   
                        risk                  55  11111         NaN   

                                            cost_cumulative  fm_active  
failure_mode            task                                            
fire_damage             CAT1/2 replacement                0       True  
                        inspection                        0       True  
                        risk                              0       True  
fungal decay _ external CAT1/2 replacement             1554       True  
                        CAT3/4 replacement                0       True  
                        inspection                      409       True  
                        risk                          11110       True  
fungal decay _ internal CAT1/2 replacement              777       True  
                        CAT3/4 replacement                0       True  
                        inspection                      403       True  
                        pole_saver_rod                    0       True  
                        risk                           5555       True  
impact                  CAT1/2 replacement                0       True  
                        inspection                        0       True  
                        risk                              0       True  
lightning               CAT1/2 replacement                0       True  
                        inspection_1                      0       True  
                        inspection_2                    398       True  
                        risk                              0       True  
termites                CAT1/2 replacement              777       True  
                        CAT3/4 replacement                0       True  
                        inspection                      414       True  
                        risk                           5555       True  
                        termite_powder                    0       True  
weathering     

In [ ]:
erc = dict()
for fm_name, fm in self.fm.items():
    erc[fm_name] = fm.expected_risk_cost_df()

df = pd.concat(erc)
df = (
    df.reset_index()
    .drop(columns="level_1")
    .rename(columns={"level_0": "failure_mode"})
)

In [14]:
comp.fm['termites'].expected_risk_cost_df()

,task,time,cost,task_active,cost_cumulative,fm_active
0,inspection,0,0,True,0,True
1,inspection,1,0,True,0,True
2,inspection,2,0,True,0,True
3,inspection,3,0,True,0,True
4,inspection,4,0,True,0,True
...,...,...,...,...,...,...
310,risk,58,0,NaN,5555,True
311,risk,59,0,NaN,5555,True
312,risk,60,0,NaN,5555,True
313,risk,61,0,NaN,5555,True


In [13]:
df['time'].max()

62

In [9]:
%%timeit

comp.expected_inspection_interval(t_min=1, t_max=10)

In [7]:
pof_sim.cancel_sim()
pof_sim = copy.copy(comp)
pof_sim.next_sim(t_end=200)

In [26]:
comp.reset()

In [30]:
pof_sim.fm['random'].timeline

{}

In [13]:
task.active

False